# Project 5 - Permanent Residency, will I get accepted?

## Libraries & Modules

In [ ]:
# General 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pickle
import operator
from collections import Counter
from itertools import product

# Made modules
import sys
sys.path.append('/Users/laurengilson/Desktop/project5')
from cleaning import *
from analysisfunctions import *

# Modeling functions
from sklearn.metrics import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV 
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # Prevents kernel from dying when running XGBoost
import xgboost as xgb

# Jupyter Formatting
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

---

# Data Collection

## PERM data from Dept. of Labour

In [ ]:
# Data available from 2008-2019 
years = list(range(2008, 2020))
perm_df = GetPermData(years)

In [ ]:
# save original imported df
perm_df.to_pickle("./perm_df.pkl")

---

## Clean PERM data columns

In [ ]:
# Initiate a new dataframe to add cleaned columns to - this will be final dataframe to merge with economic data
perm_cleaned_df = pd.DataFrame()

### Clean Y predictor column - Case Outcome

In [ ]:
perm_df['case_status'] = list(map(lambda value : value.lower(), perm_df['case_status']))

# Check various outcomes - make sure they are consistent
values, counts = np.unique(perm_df['case_status'], return_counts=True)
dict(zip(values, counts))

In [ ]:
# Remove rows where application was withdrawn, as outcome is unknown
perm_df = perm_df.drop(perm_df[perm_df['case_status'] == 'withdrawn'].index).reset_index(drop=True)

In [ ]:
# Categorise 1 into denied and everything else as 0 into a new column - assume that certified-expired is certified
perm_df["case_outcome"] = perm_df["case_status"].apply(Predictor)

In [ ]:
# Append columns needed for model to cleaned df
perm_cleaned_df['case_outcome'] = perm_df['case_outcome'].astype('category')
perm_cleaned_df['fiscal_year_of_application'] = perm_df['fiscal_year'].astype('category')

### Add Processing Center - extracted from case_number

In [ ]:
# merge case_number and case_no columns - same information currently different headers
perm_df['case_number'].fillna(perm_df['case_no'], inplace=True)

In [ ]:
# Letter at beginning of case number represents processing centre - A = Atlanta, C = Chicago
perm_df['processing_center'] = [i[0] for i in perm_df['case_number']]

In [ ]:
perm_cleaned_df['processing_center'] = perm_df['processing_center'].astype('category')

### Clean class of admission - visa which was used to enter the US

In [ ]:
perm_df['class_of_admission'] = perm_df['class_of_admission'].fillna(str('unknown')) # create unknown category for nulls
perm_df['class_of_admission'] = list(map(str, perm_df['class_of_admission']))
perm_df['class_of_admission'] = list(map(lambda x:x.lower(), perm_df['class_of_admission']))
perm_df['class_of_admission'] = [x.replace('-', '') for x in perm_df['class_of_admission']]

In [ ]:
perm_cleaned_df['class_of_admission'] = perm_df['class_of_admission'].astype('category')

### Clean country of citizenship

In [ ]:
# Merge two columns as one has a mispelling and should be the same column
perm_df['country_of_citizenship'].fillna(perm_df['country_of_citzenship'], inplace=True)

In [ ]:
# Use unknown category for nulls and remove punctuation for consistency across countries
perm_df['country_of_citizenship'] = perm_df['country_of_citizenship'].fillna(str('unknown'))
perm_df['country_of_citizenship'] = list(map(lambda x:x.lower(), perm_df['country_of_citizenship']))
perm_df['country_of_citizenship'] = perm_df['country_of_citizenship'].str.replace('(', '')
perm_df['country_of_citizenship'] = perm_df['country_of_citizenship'].str.replace(')', '')

In [ ]:
# Match country with a 3 letter identifier - makes merging with economic data easier as there are usually 
# a lot of inconsistencies with names/spellings/abbreviations - therefore, standardizing the countries to 3 letters.
perm_df['citizenship_code'] = perm_df['country_of_citizenship'].apply(CountryCode)

In [ ]:
perm_cleaned_df['country_of_citizenship'] = perm_df['citizenship_code'].astype(str)

### Add column based on whether citizenship and place of birth are the same

In [ ]:
perm_df['fw_info_birth_country'] = StringColumns(perm_df['preparer_info_emp_completed'], new_null='unknown')

In [ ]:
# Create column based on citizenship and place of birth are the same - assume they are for unknowns
citizenship_same_as_birth = list(ColValueCheck(perm_df['country_of_citizenship'], perm_df['fw_info_birth_country']))
perm_df['citizenship_same_as_birth'] = citizenship_same_as_birth

In [ ]:
perm_cleaned_df['citizenship_same_as_birth'] = perm_df['citizenship_same_as_birth'].astype('category')

### Standardize all wage requests into salaries

In [ ]:
perm_df['salary_for_job_requested'] = WageFunction(perm_df['pw_amount_9089'], perm_df['pw_unit_of_pay_9089'])

In [ ]:
perm_cleaned_df['wage_for_job'] = perm_df['salary_for_job_requested'].astype(float)

### Make groupings of 'Standard Occupation Classification'

In [ ]:
perm_df['pw_soc_code'] = list(map(str, perm_df['pw_soc_code']))
perm_df['pw_soc_code'] = [i[0:2] for i in perm_df['pw_soc_code']] # First 2 numbers represent main category of work
perm_df['pw_soc_code'] = [x.replace('na', '00') for x in perm_df['pw_soc_code']] # Make category of '00' for unknowns

In [ ]:
perm_cleaned_df['job_soc_code'] = perm_df['pw_soc_code'].astype('category')

### Add Economic sector for job

In [ ]:
perm_df['us_economic_sector'] = StringColumns(perm_df['us_economic_sector'], new_null='unclassified')

In [ ]:
perm_cleaned_df['job_economic_sector'] = perm_df['us_economic_sector'].astype(str)

### Add column determining whether the employer prepared the application or not

In [ ]:
perm_df['preparer_info_emp_completed'] = StringColumns(perm_df['preparer_info_emp_completed'])

In [ ]:
# Change to categories 0 for employer did not (n), 1 for employer did (y)
perm_df["preparer_info_emp_completed"] = perm_df["preparer_info_emp_completed"].apply(CategoriseYN)

In [ ]:
perm_cleaned_df['employer_completed_application'] = perm_df['preparer_info_emp_completed'].astype('category')

### Add Decision Date - useful for looking at applications over time

In [ ]:
perm_df['decision_date'] = pd.to_datetime(perm_df['decision_date'])
# Reduce decision date to month and year as a code - E.g 200811 = November 2008
perm_df['decision_date'] = perm_df['decision_date'].map(lambda x: 100*x.year + x.month)

In [ ]:
perm_cleaned_df['decision_month_year'] = perm_df['decision_date'].astype('category')

### Add applicant education

In [ ]:
perm_df['foreign_worker_info_education'] = StringColumns(perm_df['foreign_worker_info_education'])

In [ ]:
perm_cleaned_df['applicant_highest_education'] = perm_df['foreign_worker_info_education'].astype('category')

### Add whether applicant needs training for the job

In [ ]:
perm_df['job_info_training'] = StringColumns(perm_df['job_info_training'])

In [ ]:
# Change to 0 (not needed) and 1 (needed)
perm_df['job_info_training'] = perm_df['job_info_training'].apply(CategoriseYN)

In [ ]:
perm_cleaned_df['training_required'] = perm_df['job_info_training'].astype('category')

### Has applicant had layoff in the past six months

In [ ]:
perm_df['ri_layoff_in_past_six_months'] = StringColumns(perm_df['ri_layoff_in_past_six_months'])

In [ ]:
# Change to 0 (no) and 1 (yes)
perm_df['ri_layoff_in_past_six_months'] = perm_df['ri_layoff_in_past_six_months'].apply(CategoriseYN)

In [ ]:
perm_cleaned_df['layoff_in_past_six_months'] = perm_df['ri_layoff_in_past_six_months'].astype('category')

### Has applicant got ownership interest in the company

In [ ]:
perm_df['fw_ownership_interest'] = StringColumns(perm_df['fw_ownership_interest'])

In [ ]:
# As before 0 (no) and 1 (yes)
perm_df['fw_ownership_interest'] = perm_df['fw_ownership_interest'].apply(CategoriseYN)

In [ ]:
perm_cleaned_df['ownership_interest'] = perm_df['fw_ownership_interest'].astype('category')

### Add number of employees at company

In [ ]:
# Assume nulls as the mean value
perm_df['employer_num_employees'] = perm_df['employer_num_employees'].fillna(perm_df['employer_num_employees'].mean(axis=0))

In [ ]:
perm_cleaned_df['employer_num_employees'] = perm_df['employer_num_employees'].astype(int)

### Add applicant's state - as an abbreviation

In [ ]:
# change all states in two letter abbreviation
perm_df['fw_worker_state_abv'] = StateAbbreviation(perm_df['foreign_worker_info_state'])

In [ ]:
perm_df['fw_worker_state_abv'] = [x.replace('NONE', 'unknown') for x in perm_df['fw_worker_state_abv']] # Assume nans as n

In [ ]:
perm_cleaned_df['worker_state_abv'] = perm_df['fw_worker_state_abv'].astype('category')

### Add column of whether job is in the same state

In [ ]:
# Change job state to abbreviations - easier to merge with applicant's state
perm_df['job_state_abv'] = StateAbbreviation(perm_df['job_info_work_state'])

In [ ]:
# is job in same state - for null values assume the job is in the same state as the applicant
job_in_same_state = list(ColValueCheck(perm_df['fw_worker_state_abv'], perm_df['job_state_abv']))
perm_df['job_same_state'] = job_in_same_state

In [ ]:
perm_cleaned_df['job_same_state'] = perm_df['job_same_state'].astype('category')

### Add whether applicant has required experience for job

In [ ]:
# merge cols that should be the same according to schema
perm_df['fw_info_rel_occup_exp'].fillna(perm_df['fw_info_rel_occup_experience'], inplace = True)
perm_df['fw_info_req_experience'].fillna(perm_df['fw_info_rel_occup_exp'], inplace = True)

In [ ]:
perm_df['fw_info_req_experience'] = StringColumns(perm_df['fw_info_req_experience'])

In [ ]:
perm_cleaned_df['has_required_experience'] = perm_df['fw_info_req_experience'].astype('category')

### Save cleaned dataframe

In [ ]:
# Drop rows where citizenship is unknown - won't be able to merge economic data
perm_cleaned_df = perm_cleaned_df.drop(perm_cleaned_df[perm_cleaned_df['country_of_citizenship'] == 'NA'].index).reset_index(drop=True)

In [ ]:
# pickle clean df
perm_cleaned_df.to_pickle("./perm_clean_df.pkl")

---

## Import and clean Economic Data

In [ ]:
economic_list = ['GDP_data','employment_to_pop_ratio','gov_expenditure_education','percentage_of_immigrants','net_migration']

In [ ]:
cleaned_econ_dfs = []

for data in economic_list:
    cleaned_econ_dfs.append(CleanEconomic(data)) # add to a list - to append to PERM dataframe individually

---

## Merge PERM and Economic Data

In [ ]:
# Merge each of the economic dataframes to main PERM dataframe dependent on year of application and citizenship
for cleaned_df in cleaned_econ_dfs:
    perm_df_cleaned  = pd.merge(perm_df_cleaned, cleaned_df,  how='left', left_on=['country_of_citizenship','fiscal_year_of_application'], right_on = ['Country Code','year'])
    perm_df_cleaned.drop(['Country Code','year'], axis=1, inplace=True)

In [ ]:
# add column of which political party was in office at time of application decision - useful for analysis over time
perm_df_cleaned['political_party'] = (perm_df_cleaned['decision_month_year'].apply(PoliticalParty).astype('category'))

In [ ]:
# pickle final dataframe for analysis
perm_df_cleaned.to_pickle("./final_clean_df.pkl")
perm_df_cleaned.to_csv('full_perm_data.csv', index=False) # Save to csv - easier if working with Tableau

---

# Modelling

## Assess data

In [ ]:
perm_df_cleaned = pd.read_pickle("./final_clean_df.pkl")

In [ ]:
# Double check nulls have been dropped - cleaning process has altered these values dependent on column
perm_df_cleaned.dropna(inplace=True)

In [ ]:
# Check data imbalance
print(f" Certified: {((perm_df_cleaned.case_outcome.value_counts()[0]/perm_df_cleaned.case_outcome.count())*100).round(2)}%")
print(f" Denied: {((perm_df_cleaned.case_outcome.value_counts()[1]/perm_df_cleaned.case_outcome.count())*100).round(2)}%")

In [ ]:
perm_df_cleaned.head()

In [ ]:
perm_df_cleaned.info() # Check columns are in correct format 

In [ ]:
# Initialising a random state - to be consistent throughout
rs=27

## Dummy categorical variables

In [ ]:
# Create a list of just categorical columns in dataframe - used to get dummies
categories = []
for col in list(perm_df_cleaned.columns)[1:]:
    if perm_df_cleaned[col].dtype == 'float64' or perm_df_cleaned[col].dtype == 'int64':
        None
    else:
        categories.append(col)

In [ ]:
perm_dummied = perm_df_cleaned.join(pd.get_dummies(perm_df_cleaned[categories], drop_first=True))
perm_dummied.drop(categories, axis=1, inplace=True)

## Undersample & split data

In [ ]:
# Oversampling to 1.5million is too computationally expensive with over 500 columns - so using undersampling

In [ ]:
X = perm_dummied.drop('case_outcome', axis=1)
y = perm_dummied['case_outcome']

In [ ]:
# Split data using all of the features in the dataset - this function ensures redefining the data if less columns are 
# used more straightforward - can access the data via the dictionary and key values
all_features = SplitData(X,y)

## Random Forest - All features

In [ ]:
rf_all = RandomForestClassifier(random_state=rs).fit(all_features['X_train'], all_features['y_train'])
rf_all_pred = rf_all.predict(all_features['X_val'])

In [ ]:
ErrorMetrics(rf_all, "Random Forest, All Features", all_features['X_train'], all_features['y_train'])

In [ ]:
rf_all_cm = confusion_matrix(all_features['y_val'], rf_all_pred)
PrintConfusionMatrix(rf_all_cm)

In [ ]:
ROC_AUC(rf_all, "Random Forest All features", all_features['y_val'], all_features['X_val'])

## Decision Tree - All features

In [ ]:
dec_tree = DecisionTreeClassifier(random_state=rs).fit(all_features['X_train'], all_features['y_train'])
y_pred_dec_tree = dec_tree.predict(all_features['X_val'])

In [ ]:
ErrorMetrics(dec_tree, 'Decision Tree, All features', all_features['X_train'], all_features['y_train'])

In [ ]:
dec_tree_cm = confusion_matrix(all_features['y_val'], y_pred_dec_tree)
PrintConfusionMatrix(dec_tree_cm)

In [ ]:
ROC_AUC(dec_tree, 'Decision Tree, All features', all_features['y_val'], all_features['X_val'])

### Feature Importance

In [ ]:
# Get feature importance, in order and with correct column names
features = list(dec_tree.feature_importances_.round(3))
column_names = list(perm_dummied.columns.drop('case_outcome'))
importance = dict(zip(column_names, features))
dec_tree_ordered_importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
dec_tree_ordered_importance[:11]

In [ ]:
# Determine top feature importance and separate list
most_important = [k for k,v in importance.items() if v>=0.004]
most_important

## Grid Search to find optimal parameters for Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=rs)
# Currently default parameters used in random forest classifier
rf.get_params()

In [ ]:
# Set various parameters to test
params_to_test = {
    'bootstrap': [True],
    'max_depth': [2, 5, 10],
    'max_features':[2,3],
    'min_samples_leaf':[3,4,5],
    'min_samples_split':[8,10,12],
    'n_estimators':[100,200,500]
}

In [ ]:
grid_search = GridSearchCV(estimator=rf, param_grid=params_to_test, cv=3, n_jobs=-1, verbose=2)

In [ ]:
# Takes approx 45 minutes
grid_search.fit(all_features['X_train'], all_features['y_train'])

In [ ]:
best_rf_params = grid_search.best_params_
best_rf_params

```{'bootstrap': True,
    'max_depth' : 10,
    'max_features' : 3, 
    'min_samples_leaf' : 3,
    'min_samples_split' : 10, 
    'n_estimators' : 500}```

In [ ]:
best_rf_estimator = grid_search.best_estimator_
best_rf_estimator

```RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=10, max_features=3, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=3, min_samples_split=10,
                        min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
                        oob_score=False, random_state=27, verbose=0, warm_start=False)```

## Random Forest - All features & Optimal Parameters

In [ ]:
rf_all_best_params = best_rf_estimator.fit(all_features['X_train'], all_features['y_train'])
rf_all_best_pred = rf_all_best_params.predict(all_features['X_val'])

In [ ]:
ErrorMetrics(rf_all_best_params, 'Random Forest, All features, Best Parameters', all_features['X_train'], all_features['y_train'])

In [ ]:
rf_all_best_params_cm = confusion_matrix(all_features['y_val'], rf_all_best_pred)
PrintConfusionMatrix(rf_all_best_params_cm)

In [ ]:
ROC_AUC(rf_all_best_params, "Random Forest, All features, Best Parameters", all_features['y_val'], all_features['X_val'])

## Random Forest - Top features & Optimal Parameters

In [ ]:
# Resplit data with most important features
most_imp_X = perm_dummied[most_important]
most_imp_y = perm_dummied['case_outcome']

most_important_features = SplitData(most_imp_X, most_imp_y)

In [ ]:
rf_top_feats = best_rf_estimator.fit(most_important_features['X_train'], most_important_features['y_train'])
y_pred_top_feats = rf_top_feats.predict(most_important_features['X_val'])

In [ ]:
ErrorMetrics(rf_top_feats, 'Random Forest, Top Features', most_important_features['X_train'], most_important_features['y_train'])

In [ ]:
rf_top_feats_cm = confusion_matrix(most_important_features['y_val'], y_pred_top_feats)
PrintConfusionMatrix(rf_top_feats_cm)

In [ ]:
ROC_AUC(rf_top_feats, "Random Forest, Top Features", most_important_features['y_val'], most_important_features['X_val'])

## Naive Bayes - All features

In [ ]:
naive_bayes = GaussianNB().fit(all_features['X_train'], all_features['y_train'])
y_naive_pred = naive_bayes.predict(all_features['X_val'])

In [ ]:
ErrorMetrics(naive_bayes, 'Naive Bayes All Features', all_features['X_train'], all_features['y_train'])

In [ ]:
nb_cm = confusion_matrix(all_features['y_val'], y_naive_pred)
PrintConfusionMatrix(nb_cm, ['Certified', 'Denied'])

In [ ]:
ROC_AUC(naive_bayes, "Naive Bayes All features", all_features['y_val'], all_features['X_val'])

## Naive Bayes - Top features

In [ ]:
naive_bayes_most_imp = GaussianNB().fit(most_important_features['X_train'], most_important_features['y_train'])
y_naive_pred_most_imp = naive_bayes_most_imp.predict(most_important_features['X_val'])

In [ ]:
ErrorMetrics(naive_bayes_most_imp, 'Naive Bayes Most Imp Features', most_important_features['X_train'], most_important_features['y_train'])

In [ ]:
nb_most_imp_cm = confusion_matrix(most_important_features['y_val'], y_naive_pred_most_imp)
PrintConfusionMatrix(nb_most_imp_cm, ['Certified', 'Denied'])

In [ ]:
ROC_AUC(naive_bayes_most_imp, "Naive Bayes Most imp features", most_important_features['y_val'], most_important_features['X_val'])

## Grid Search for XGBoost

In [ ]:
# define parameters to search for XGBoost
gridsearch_params = {
        'max_depth': [3, 4],
        'learning_rate':[0.02, 0.05],
        'min_child_weight': [1, 5],
        'gamma': [0.5, 1, 2],
        'subsample': [0.6, 0.8],
        'colsample_bytree': [0.6, 0.8],
        }

In [ ]:
xgb_grid = xgb.XGBClassifier(n_estimator=50000, objective='binary:logistic')

In [ ]:
grid_search_xgb = GridSearchCV(estimator = xgb_grid, param_grid = gridsearch_params, cv=3, n_jobs = -1, verbose=2)

In [ ]:
# Runtime: approx 4hours
grid_search_xgb.fit(all_features['X_train'], all_features['y_train'])

In [ ]:
grid_search_xgb.best_params_

```{'colsample_bytree': 0.8,
 'gamma': 1,
 'learning_rate': 0.05,
 'max_depth': 4,
 'min_child_weight': 1,
 'subsample': 0.8}
 ```

## XGBoost - All features & Optimal Parameters

In [ ]:
best_params_xgb = xgb.XGBClassifier(n_estimators=30000,
                                   max_depth=4,
                                   objective='binary:logistic',
                                   gamma=1,
                                   learning_rate=0.05,
                                   subsample=0.8,
                                   min_child_weight=1,
                                   colsample_bytree=0.8
                                   )

In [ ]:
eval_set_all = [(all_features['X_train'],all_features['y_train']), (all_features['X_val'], all_features['y_val'])]

In [ ]:
best_params_xgb.fit(all_features['X_train'],all_features['y_train'], 
                    eval_set=eval_set_all,
                    eval_metric='error', # error = 1 - accuracy 
                    early_stopping_rounds=20, # stop after 20 rounds if no improvement in error
                    verbose=True
                   )

In [ ]:
best_params_pred = best_params_xgb.predict(all_features['X_val'])

In [ ]:
xgb_cm = confusion_matrix(all_features['y_val'], best_params_pred)
PrintConfusionMatrix(xgb_cm)

In [ ]:
# ErrorMetrics runs too slowly for XGBoost - manual print instead
print(f"Accuracy: {accuracy_score(best_params_xgb.predict(all_features['X_val'], all_features['y_val'])}")
print(f"Precision: {precision_score(best_params_xgb.predict(all_features['X_val'], all_features['y_val'])}")
print(f"Recall: {recall_score(best_params_xgb.predict(all_features['X_val'], all_features['y_val'])}")
print(f"F1: {f1_score(best_params_xgb.predict(all_features['X_val'], all_features['y_val'])}")

### Feature Importance

In [ ]:
# Compare with decision tree importances to determine which are most/least for interpretation
xgb.plot_importance(best_params_xgb, max_num_features=10)
xgb.plot_importance(best_params_xgb, max_num_features=10, importance_type='gain')

In [ ]:
xgb_features = list(best_params_xgb.feature_importances_.round(3))
xgb_importance = dict(zip(column_names, xgb_features))
xgb_ordered_importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
xgb_ordered_importance[:11]

## XGBoost  - Top features & Optimal Parameters

In [ ]:
best_params_top_feats = xgb.XGBClassifier(n_estimators=30000,
                                   max_depth=4,
                                   objective='binary:logistic',
                                   gamma=1,
                                   learning_rate=0.05,
                                   subsample=0.8,
                                   min_child_weight=1,
                                   colsample_bytree=0.8
                                   )

In [ ]:
new_eval_most_imp = [(most_important_features['X_train'],most_important_features['y_train']), (most_important_features['X_val'], most_important_features['y_val'])]

In [ ]:
best_params_top_feats.fit(most_important_features['X_train'],most_important_features['y_train'], 
                          eval_set=new_eval_most_imp,
                          eval_metric='error', # error = 1 - accuracy 
                          early_stopping_rounds=20, # stop after 20 rounds if no improvement in error
                          verbose=True
                          )

In [ ]:
xgb_top_feats_pred = best_params_top_feats.predict(most_important_features['X_val'])

In [ ]:
xgb_cm = confusion_matrix(most_important_features['y_val'], xgb_top_feats_pred)
PrintConfusionMatrix(xgb_cm)

In [ ]:
print(f"Accuracy: {accuracy_score(best_params_top_feats.predict(most_important_features['X_val'], most_important_features['y_val'])}")
print(f"Precision: {precision_score(best_params_top_feats.predict(most_important_features['X_val'], most_important_features['y_val'])}")
print(f"Recall: {recall_score(best_params_top_feats.predict(most_important_features['X_val'], most_important_features['y_val'])}")
print(f"F1: {f1_score(best_params_top_feats.predict(most_important_features['X_val'], most_important_features['y_val'])}")

In [ ]:
ROC_AUC(best_params_top_feats, "XGBoost Top features", most_important_features['y_val'], most_important_features['X_val'])

## Try Voting Classifier - Random Forest & Naive Bayes - All features

In [ ]:
X_vote = perm_dummied.drop('case_outcome', axis=1)
y_vote = perm_dummied['case_outcome']
voting = SplitData(X_vote, y_vote)

In [ ]:
# define classifiers to vote against
clf1 = best_rf_estimator
clf2 = GaussianNB()

### Hard Voting

In [ ]:
hard_vote = VotingClassifier(estimators=[('RF', clf1), ('naive_bayes', clf2)], voting='hard').fit(voting['X_train'], voting['y_train'])
hard_pred = hard_vote.predict(voting['X_val'])

In [ ]:
hard_vote_cm = confusion_matrix(voting['y_val'], hard_pred)
PrintConfusionMatrix(hard_vote_cm)

### Soft Voting

In [ ]:
soft_vote = VotingClassifier(estimators=[('RF', clf1), ('naive_bayes', clf2)], voting='soft').fit(voting['X_train'], voting['y_train'])
soft_pred = soft_vote.predict(voting['X_val'])

In [ ]:
soft_vote_cm = confusion_matrix(voting['y_val'], soft_pred)
print_confusion_matrix(soft_vote_cm)

In [ ]:
ROC_AUC(soft_vote, "Soft Vote", voting['y_val'], voting['X_val'])

### Weighted Voting

In [ ]:
# Try combinations of weightings using itertools product to find optimal pairing
pairing = []
accuracy = []

for item in list(product('123456789', repeat=2)):
    weights = [int(item[0]), int(item[1])]
    
    weight_vote = VotingClassifier(estimators=[('RF', clf1), ('naive_bayes', clf2)], voting='soft', weights=weights, flatten_transform=True).fit(voting['X_train'], voting['y_train'])
    weight_pred = weight_vote.predict(voting['X_val'])
    
    pairing.append(weights)
    accuracy.append(weight_vote.score(voting['X_train'], voting['y_train']))
    

optimal_pair = pairing[accuracy.index(max(accuracy))]

print(f"Pairing: {optimal_pair}, Accuracy: {accuracy.index(max(accuracy))}")

In [ ]:
weight_vote = VotingClassifier(estimators=[('RF', clf1), ('naive_bayes', clf2)], voting='soft', weights=optimal_pair, flatten_transform=True).fit(voting['X_train'], voting['y_train'])
weight_pred = weight_vote.predict(voting['X_val'])

In [ ]:
weight_vote_cm = confusion_matrix(voting['y_val'], weight_pred)
PrintConfusionMatrix(weight_vote_cm)

In [ ]:
ROC_AUC(weight_vote, "VotingWeighted", voting['y_val'], voting['X_val'])

---

# Test model on hold out data

**Conclusions:** 
- Best performing model - XGBoost with top features and optimal parameters
- **Accuracy:** 73.3%
- **Precision:** 70.0%
- **AUC:** 0.816
- **FPR:** 14.7% (Total) 29.4% (Denied Cases)

In [ ]:
final_preds = best_params_top_feats.predict(most_important_features['X_hold_out'])

In [ ]:
final_preds = confusion_matrix(most_important_features_features['y_hold_out'], final_preds)
PrintConfusionMatrix(final_cm)

In [ ]:
print(f"Accuracy: {accuracy_score(best_params_top_feats.predict(most_important_features['X_hold_out'], most_important_features['y_hold_out']))}")
print(f"Precision: {precision_score(best_params_top_feats.predict(most_important_features['X_hold_out'], most_important_features['y_hold_out']))}")

---

# Compare Feature Importances

In [ ]:
# Look at which features appear in the top 10 features for Dec Trees & XGBoost
dec_tree_top_ten = dec_tree_ordered_importance[:11]
dec_tree_lowest_ten = dec_tree_ordered_importance[-10:]
xgb_top_ten = xgb_ordered_importance[:11]
xgb_lowest_ten = xgb_ordered_importance[-10:]

In [ ]:
# Find common most influential features
matching_highest = []

for dt in dec_tree_top_ten:
    for xgb in xgb_top_ten:
        if dt[0] == xgb[0]:
            matching_highest.append(dt[0])

matching_highest

In [ ]:
# Find common least influential features
matching_lowest = []

for dt in dec_tree_lowest_ten:
    for xgb in xgb_lowest_ten:
        if dt[0] == xgb[0]:
            matching_lowest.append(dt[0])

matching_lowest

---

# Look at applications & outcomes over time

## Per year

In [ ]:
rejections = pd.DataFrame(perm_df.groupby('fiscal_year_of_application')['case_outcome'].value_counts())
rejections.head()

In [ ]:
# Calculate percentage of rejections per year
percentage_of_rejecs = []
for i in range(0, 24, 2):
    percentage_of_rejecs.append(round((rejections.iloc[i+1][0]/(rejections.iloc[i][0]+rejections.iloc[i+1][0])*100),2))

In [ ]:
year_list = list(range(2008, 2020))
year_list

In [ ]:
# Plot rejection rate over time (year)
plt.figure(figsize=(8,5))
plt.plot(year_list, percentage_of_rejecs)
plt.xlabel('Year')
plt.ylabel('Percentage of Application Rejections');

In [ ]:
# Plot raw number of applications over time
plt.figure(figsize=(8,5))
plt.plot(year_list[:-1], list(application['case_outcome'][:-1]))
plt.xlabel('Year')
plt.ylabel('Number of Applications');

## Per month

### Rejection Number & Rate

In [ ]:
each_month = pd.DataFrame(perm_df.groupby('decision_month_year')['case_outcome'].value_counts())
each_month.head()

In [ ]:
# Percentage of decisions in each month being rejections
month_percentage = []
for i in range(0, (perm_df['decision_month_year'].nunique()*2), 2):
    month_percentage.append(round((each_month.iloc[i+1][0]/(each_month.iloc[i][0]+each_month.iloc[i+1][0])*100),2))

In [ ]:
# Total Number of rejections per month 
month_number = []
for i in range(0, (perm_df['decision_month_year'].nunique()*2), 2):
    month_number.append(round((each_month.iloc[i+1][0])))

In [ ]:
# Make months unto more readable terms for graphs
months = []

for month in sorted(list(perm_df['decision_month_year'].unique())):
    date = datetime.datetime.strptime(str(month), '%Y%m')
    months.append(date.strftime("%b %Y"))

In [ ]:
# Include political party information 
democratic_start = months[months.index('Jan 2009')] 
democratic_end = months[months.index('Jan 2017')]

In [ ]:
# Plot percentage of rejections 
plt.figure(figsize=(20,5))
plt.plot(months, month_percentage)
plt.xticks(months[::12], rotation=45)
plt.xlabel('Year')
plt.ylabel('Percentage of Application Rejections')

# Shade political party in office at time - democratic - blue, republican - red
plt.axvspan(months[0], democratic_start, alpha=0.2, color='r')
plt.axvspan(democratic_start, democratic_end, alpha=0.2, color='b')
plt.axvspan(democratic_end, months[-1], alpha=0.2, color='r')

for month in months[::12]:
    plt.axvline(month, color='0.5', zorder=0, linestyle=':');

In [ ]:
# Raw number of rejections 
plt.figure(figsize=(20,5))
plt.plot(months, month_number)
plt.xticks(months[::12], rotation=45)
plt.xlabel('Fiscal Year')
plt.ylabel('Number of Application Rejections')

# Shade political party in office at time
plt.axvspan(months[0], democratic_start, alpha=0.2, color='r')
plt.axvspan(democratic_start, democratic_end, alpha=0.2, color='b')
plt.axvspan(democratic_end, months[-1], alpha=0.2, color='r')

for month in months[::12]:
    plt.axvline(month, color='0.5', zorder=0, linestyle=':');

### Number of applications per month

In [ ]:
application_month = perm_df.groupby(['decision_month_year']).agg('count')
application_month.head()

In [ ]:
# Plot total number of applications per month - can compare with percentage of rejections see if there's correlation 
plt.figure(figsize=(20,5))
plt.plot(months, list(application_month['case_outcome']))
plt.xticks(new_year, rotation=45)
plt.xlabel('Year')
plt.ylabel('Number of Applications')

# Shade political party in office at time
plt.axvspan(months[0], democratic_start, alpha=0.2, color='r')
plt.axvspan(democratic_start, democratic_end, alpha=0.2, color='b')
plt.axvspan(democratic_end, months[-1], alpha=0.2, color='r');

---